In [1]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [2]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [4]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [5]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transform_data(self):
        # Load the data
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets with randomization (0.75, 0.25) split.
        train, test = train_test_split(data, test_size=0.25, random_state=42)

        # Apply StandardScaler
        scaler = StandardScaler()
        train_scaled = scaler.fit_transform(train)
        test_scaled = scaler.transform(test)

        # Apply PCA (Principal Component Analysis)
        #pca = PCA(n_components=2)  # You can specify the number of components as needed
        #train_pca = pca.fit_transform(train_scaled)
        #test_pca = pca.transform(test_scaled)

        # Save the transformed data
        train = pd.DataFrame(train_scaled)
        test = pd.DataFrame(test_scaled)

        train.to_csv(os.path.join(self.config.root_dir, "train_pca.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test_pca.csv"), index=False)

        logger.info("Transformed data with StandardScaler and PCA and saved to CSV files with randomization")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_data()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'